# Bayesian Optimization with ENTMOOT

This notebook describes how to perform Bayesian optimization with a simple test function.

In [ ]:
from entmoot import Enting, ProblemConfig, PyomoOptimizer
from entmoot.benchmarks import build_reals_only_problem, eval_reals_only_testfunc
import numpy as np

In [ ]:
# define problem
problem_config = ProblemConfig(rnd_seed=73)
build_reals_only_problem(problem_config)
# sample data
N_init = 10
train_x = problem_config.get_rnd_sample_list(num_samples=N_init)
train_y = eval_reals_only_testfunc(train_x)

# create enting object
# this defines the surrogate model
params = {"unc_params": {"dist_metric": "l1", "acq_sense": "exploration"}}
enting = Enting(problem_config, params=params)

# create optimizer object
params_pyomo = {"solver_name": "gurobi", "verbose": False}
opt_pyo = PyomoOptimizer(problem_config, params=params_pyomo)

We now run the BO loop `N_iterations` times. At each iteration, we fit the surrogate to the data, then use the optimizer to propose a new experiment.

In [ ]:
N_iterations = 40
for itr in range(N_iterations):
    enting.fit(train_x, train_y)
    result = opt_pyo.solve(enting)

    next_x = np.reshape(result.opt_point, (1, -1))
    next_y = eval_reals_only_testfunc(next_x)

    # update the training data
    train_x = np.concatenate([train_x, next_x], axis=0)
    train_y = np.concatenate([train_y, next_y], axis=0)

In [ ]:
import matplotlib.pyplot as plt
iterations = np.arange(N_init, N_init + N_iterations + 1)
plt.step(iterations, np.minimum.accumulate(train_y)[N_init-1:])